# Processing
Process and merge data files. Restrict from July 1963 to July 2024.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [2]:
# Load the datasets
industry_portfolios = pd.read_csv('data/10_Industry_Portfolios.csv', index_col=0, header=6, nrows=1177, date_format='%Y%m', parse_dates=True) # 1177 rows for Average Value Weighted Returns -- Monthly table
ff_factors = pd.read_csv('data/F-F_Research_Data_Factors.csv', index_col=0, header=2, nrows=1177, date_format='%Y%m', parse_dates=True)

## Exploration

In [3]:
display(industry_portfolios.head())
display(ff_factors.head())

display(industry_portfolios.tail())
display(ff_factors.tail())

,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other
1926-07-01,1.45,15.55,4.69,-1.18,2.90,0.83,0.11,1.77,7.04,2.13
1926-08-01,3.97,3.68,2.81,3.47,2.66,2.17,-0.71,4.25,-1.69,4.35
1926-09-01,1.14,4.80,1.15,-3.39,-0.38,2.41,0.21,0.69,2.04,0.29
1926-10-01,-1.24,-8.23,-3.63,-0.78,-4.58,-0.11,-2.29,-0.57,-2.63,-2.84
1926-11-01,5.20,-0.19,4.10,0.01,4.71,1.63,6.43,5.42,3.71,2.11


,Mkt-RF,SMB,HML,RF
1926-07-01,2.96,-2.56,-2.43,0.22
1926-08-01,2.64,-1.17,3.82,0.25
1926-09-01,0.36,-1.40,0.13,0.23
1926-10-01,-3.24,-0.09,0.70,0.32
1926-11-01,2.53,-0.10,-0.51,0.31


,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other
2024-03-01,2.99,-5.56,4.48,10.27,2.36,3.08,2.68,2.18,7.03,4.50
2024-04-01,-1.54,-0.96,-3.87,-0.64,-4.41,-6.51,-4.72,-5.86,1.57,-4.56
2024-05-01,1.38,-0.75,2.94,-0.48,8.09,3.74,2.47,3.21,8.01,3.27
2024-06-01,-2.26,6.10,-1.82,-2.10,7.57,-0.60,3.53,2.93,-4.23,-0.13
2024-07-01,4.76,11.55,3.44,2.02,-1.73,3.85,0.55,0.84,6.36,6.22


,Mkt-RF,SMB,HML,RF
2024-03-01,2.83,-2.51,4.21,0.43
2024-04-01,-4.67,-2.39,-0.52,0.47
2024-05-01,4.34,0.78,-1.66,0.44
2024-06-01,2.77,-3.06,-3.31,0.41
2024-07-01,1.24,6.84,5.70,0.45


## Merging and Cleaning

In [4]:
# Merge on date index
df = industry_portfolios.merge(ff_factors, left_index=True, right_index=True)

# Restrict from July 1963 to July 2024
df = df.loc['1963-07':'2024-07']

display(df.head(1))
display(df.tail(1))

,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other,Mkt-RF,SMB,HML,RF
1963-07-01,-0.48,-0.07,-1.39,2.3,-0.68,-0.25,-1.05,0.57,0.81,-1.59,-0.39,-0.45,-0.97,0.27


,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other,Mkt-RF,SMB,HML,RF
2024-07-01,4.76,11.55,3.44,2.02,-1.73,3.85,0.55,0.84,6.36,6.22,1.24,6.84,5.7,0.45


# Estimate Alpha and Beta w/CAPM

Estimate the alphas and betas of the 10 industry portfolios according to the CAPM. Run the regression
$$
r_{it} - r_{ft} = \alpha_i + \beta_i(r_{Mt}-r_{ft}) + \varepsilon_{it}
$$
For each industry portfolio $i$, where subscript $t$ indicates a monthly frequency. Compute the standard errors for your estimated coefficients:

i) assuming independently, identically distributed (iid) errors,

ii) allowing for heteroskedasticity (i.e., White standard errors),

iii) allowing for serial correlation (i.e., Newey-West standard errors with six lags). 

Report your regression in a well-formatted table.

In [5]:
excess_returns = df.iloc[:, :-4].subtract(df['RF'], axis=0) # Subtract risk-free rate from industry portfolios returns
display(excess_returns.head())

,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other
1963-07-01,-0.75,-0.34,-1.66,2.03,-0.95,-0.52,-1.32,0.30,0.54,-1.86
1963-08-01,4.62,6.29,5.94,3.69,4.88,4.03,6.17,9.31,3.95,5.19
1963-09-01,-1.96,-0.52,-1.05,-3.92,-0.13,2.09,0.66,-4.34,-2.77,-3.45
1963-10-01,2.37,10.42,2.25,-0.62,8.01,3.14,0.22,3.09,-0.96,1.10
1963-11-01,-1.39,-5.44,0.03,-1.42,-0.56,3.87,-1.52,-1.92,-1.29,-0.17


In [6]:
def CAPM(excess_returns):
    results = pd.DataFrame(index=excess_returns.columns, columns=['alpha', 'beta','iid_error_alpha', 'iid_error_beta', 'white_std_error_alpha', 'white_std_error_beta', 'newey_west_std_error_alpha', 'newey_west_std_error_beta'])
    
    for portfolio in excess_returns.columns:
        X = df['Mkt-RF'] # Market excess return
        y = excess_returns[portfolio]
        
        X = sm.add_constant(X)
        
        model = sm.OLS(y, X).fit()

        results.loc[portfolio, 'alpha'] = model.params[0]
        results.loc[portfolio, 'beta'] = model.params[1]

        # IID standard errors
        results.loc[portfolio, 'iid_error_alpha'] = model.bse[0]
        results.loc[portfolio, 'iid_error_beta'] = model.bse[1]

        # White standard errors
        white_model = model.get_robustcov_results(cov_type='HC0')
        results.loc[portfolio, 'white_std_error_alpha'] = white_model.bse[0]
        results.loc[portfolio, 'white_std_error_beta'] = white_model.bse[1]

        # Newey-West standard errors
        newey_west_model = model.get_robustcov_results(cov_type='HAC', maxlags=6)
        results.loc[portfolio, 'newey_west_std_error_alpha'] = newey_west_model.bse[0]
        results.loc[portfolio, 'newey_west_std_error_beta'] = newey_west_model.bse[1]

    return results

In [7]:
alpha_beta_results = CAPM(excess_returns)
display(alpha_beta_results)

/var/folders/3_/5yqvf0js6kn57hkj_5zvnlf40000gn/T/ipykernel_41312/2665581064.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  results.loc[portfolio, 'alpha'] = model.params[0]
/var/folders/3_/5yqvf0js6kn57hkj_5zvnlf40000gn/T/ipykernel_41312/2665581064.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  results.loc[portfolio, 'beta'] = model.params[1]
/var/folders/3_/5yqvf0js6kn57hkj_5zvnlf40000gn/T/ipykernel_41312/2665581064.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To 

,alpha,beta,iid_error_alpha,iid_error_beta,white_std_error_alpha,white_std_error_beta,newey_west_std_error_alpha,newey_west_std_error_beta
NoDur,0.208606,0.774248,0.090146,0.019939,0.090979,0.026149,0.104833,0.037208
Durbl,-0.090251,1.239916,0.174164,0.038523,0.164176,0.054173,0.165909,0.066132
Manuf,0.007332,1.036718,0.066376,0.014681,0.067351,0.018932,0.071643,0.022435
Enrgy,0.174194,0.86085,0.173839,0.038451,0.174575,0.051082,0.18946,0.060194
HiTec,0.0238,1.22937,0.119044,0.026331,0.119397,0.030475,0.129291,0.043655
Telcm,-0.008706,0.774718,0.116526,0.025774,0.116738,0.030474,0.127332,0.036863
Shops,0.125259,0.999918,0.094932,0.020998,0.094102,0.028347,0.102715,0.032005
Hlth,0.218939,0.821395,0.114494,0.025324,0.113146,0.035087,0.113271,0.040333
Utils,0.161408,0.525928,0.123151,0.027239,0.125278,0.034222,0.12085,0.038356
Other,-0.042532,1.097765,0.074789,0.016542,0.075828,0.019682,0.085261,0.025981


In [8]:
alpha_beta_results.to_csv('alpha_beta_results.csv')

# Calculating Alpha and Beta w/Fama-French 3-Factor Model
$$
r_{it} - r_{ft} = \alpha_i + \beta_i(r_{Mt} - r_{ft}) + s_iSMB_t + h_iHML_t + \varepsilon_{it}
$$

In [9]:
def fama_french_3_factor_model(excess_returns):
    results = pd.DataFrame(index=excess_returns.columns, columns=['alpha', 'beta_mkt', 'beta_s', 'beta_h', 'iid_error_alpha', 'iid_error_beta_mkt', 'iid_error_beta_smb', 'iid_error_beta_hml', 'white_std_error_alpha', 'white_std_error_beta_mkt', 'white_std_error_beta_smb', 'white_std_error_beta_hml', 'newey_west_std_error_alpha', 'newey_west_std_error_beta_mkt', 'newey_west_std_error_beta_smb', 'newey_west_std_error_beta_hml'])

    for portfolio in excess_returns.columns:
        X = df[['Mkt-RF', 'SMB', 'HML']]
        y = excess_returns[portfolio]
        X = sm.add_constant(X)

        model = sm.OLS(y, X).fit()

        results.loc[portfolio, 'alpha'] = model.params[0]
        results.loc[portfolio, 'beta_mkt'] = model.params[1]
        results.loc[portfolio, 'beta_s'] = model.params[2]
        results.loc[portfolio, 'beta_h'] = model.params[3]

        # IID standard errors
        results.loc[portfolio, 'iid_error_alpha'] = model.bse[0]
        results.loc[portfolio, 'iid_error_beta_mkt'] = model.bse[1]
        results.loc[portfolio, 'iid_error_beta_smb'] = model.bse[2]
        results.loc[portfolio, 'iid_error_beta_hml'] = model.bse[3]

        # White standard errors
        white_model = model.get_robustcov_results(cov_type='HC0')
        results.loc[portfolio, 'white_std_error_alpha'] = white_model.bse[0]
        results.loc[portfolio, 'white_std_error_beta_mkt'] = white_model.bse[1]
        results.loc[portfolio, 'white_std_error_beta_smb'] = white_model.bse[2]
        results.loc[portfolio, 'white_std_error_beta_hml'] = white_model.bse[3]

        newey_west_model = model.get_robustcov_results(cov_type='HAC', maxlags=6)
        results.loc[portfolio, 'newey_west_std_error_alpha'] = newey_west_model.bse[0]                     
        results.loc[portfolio, 'newey_west_std_error_beta_mkt'] = newey_west_model.bse[1]
        results.loc[portfolio, 'newey_west_std_error_beta_smb'] = newey_west_model.bse[2]
        results.loc[portfolio, 'newey_west_std_error_beta_hml'] = newey_west_model.bse[3]

    return results

In [10]:
fama_french_results = fama_french_3_factor_model(excess_returns)
display(fama_french_results)

TypeError: fama_french_3_factor_model() takes 1 positional argument but 2 were given

In [ ]:
fama_french_results.to_csv('fama_french_results.csv') 